In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [2]:
import os
contenido_proyecto = os.listdir('/content/drive/My Drive/Proyecto de Grado G')
print(contenido_proyecto)

['Seguimiento 1204', 'Anteproyecto.gdoc', 'Roadtrip to thesis.gdoc', 'Anteproyecto   DLG.gdoc', 'FundusImages', 'Deep learning model for binary classification for glaucoma from eye fundus images.gdoc', 'Contours', 'ProcessedImages_cup_Exp1', 'ProcessedImages_cup_Exp2', 'patient_data_os.xlsx', 'patient_data_od.xlsx']


In [5]:
import os
import cv2
import numpy as np

# Lista completa de IDs de pacientes
pacientes_ids = [
    "002", "004", "005", "006", "007", "008", "009", "010", "013", "014",
    "015", "016", "018", "019", "020", "021", "023", "024", "025", "026",
    "027", "028", "029", "030", "031", "032", "033", "034", "035", "036",
    "037", "038", "039", "041", "042", "044", "045", "046", "047", "048",
    "050", "051", "053", "055", "056", "057", "062", "064", "065", "066",
    "067", "068", "069", "072", "073", "074", "077", "079", "081", "084",
    "087", "088", "089", "090", "091", "092", "093", "096", "098", "101",
    "102", "104", "105", "106", "107", "108", "112", "113", "114", "116",
    "119", "120", "121", "122", "123", "124", "125", "126", "127", "128",
    "129", "130", "131", "132", "133", "134", "135", "136", "137", "138",
    "139", "140", "141", "142", "143", "144", "146", "147", "149", "150",
    "151", "152", "153", "154", "155", "156", "157", "158", "159", "160",
    "162", "163", "164", "165", "166", "168", "169", "170", "171", "172",
    "174", "175", "176", "177", "178", "179", "180", "181", "182", "183",
    "184", "186", "187", "188", "189", "190", "191", "192", "193", "194",
    "195", "196", "197", "198", "199", "200", "201", "202", "203", "204",
    "205", "206", "207", "208", "209", "210", "211", "212", "213", "214",
    "215", "216", "217", "218", "219", "220", "221", "222", "223", "224",
    "225", "227", "228", "229", "230", "231", "232", "233", "234", "235",
    "236", "237", "238", "239", "240", "241", "242", "243", "244", "245",
    "246", "248", "249", "250", "251", "252", "253", "254", "255", "256",
    "257", "258", "259", "260", "261", "262", "264", "265", "266", "267",
    "268", "269", "270", "271", "272", "273", "274", "275", "276", "277",
    "280", "281", "282", "283", "284", "285", "286", "287", "288", "289",
    "290", "291", "292", "293"
]

# Funciones para procesar las imágenes
def leer_coordenadas(archivo_txt):
    coordenadas = []
    with open(archivo_txt, 'r') as f:
        for linea in f:
            x, y = map(float, linea.strip().split())
            coordenadas.append((x, y))
    return np.array(coordenadas, dtype=np.float32)

def encontrar_circulo_minimo(contorno):
    (x, y), radio = cv2.minEnclosingCircle(contorno)
    return int(x), int(y), int(radio)

def recortar_y_redimensionar(imagen, x, y, radio, margen=0.2, dimension_final=(224, 224)):
    radio_ajustado = int(radio * (1 + margen))
    x_inicio = max(x - radio_ajustado, 0)
    y_inicio = max(y - radio_ajustado, 0)
    x_fin = min(x + radio_ajustado, imagen.shape[1])
    y_fin = min(y + radio_ajustado, imagen.shape[0])
    roi = imagen[y_inicio:y_fin, x_inicio:x_fin]
    imagen_redimensionada = cv2.resize(roi, dimension_final, interpolation=cv2.INTER_AREA)
    return imagen_redimensionada

# Función principal para procesar las imágenes
def procesar_imagenes(directorio_imagenes, directorio_coordenadas, directorios_salida):
    for directorio_salida in directorios_salida.values():
        if not os.path.exists(directorio_salida):
            os.makedirs(directorio_salida)

    for paciente_id in pacientes_ids:
        for ojo in ['OD', 'OS']:
            for experto in ['exp1', 'exp2']:
                nombre_archivo_base = f"RET{paciente_id}{ojo}"
                archivo_imagen = os.path.join(directorio_imagenes, f"{nombre_archivo_base}.jpg")
                archivo_coordenadas = os.path.join(directorio_coordenadas, f"{nombre_archivo_base}_disc_{experto}.txt")

                if os.path.exists(archivo_imagen) and os.path.exists(archivo_coordenadas):
                    print(f"Procesando: {archivo_imagen} con coordenadas de: {archivo_coordenadas}")
                    imagen = cv2.imread(archivo_imagen)
                    imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)

                    contorno = leer_coordenadas(archivo_coordenadas)
                    x, y, radio = encontrar_circulo_minimo(contorno)
                    imagen_procesada = recortar_y_redimensionar(imagen, x, y, radio)

                    nombre_salida = f"{nombre_archivo_base}_reg_disc_{experto}.jpg"
                    cv2.imwrite(os.path.join(directorios_salida[experto], nombre_salida), cv2.cvtColor(imagen_procesada, cv2.COLOR_RGB2BGR))
                    print(f"Imagen guardada en {directorios_salida[experto]}: {nombre_salida}")

# Definición de directorios base y de salida
directorio_base = '/content/drive/My Drive/Proyecto de Grado G'
directorio_imagenes = os.path.join(directorio_base, 'FundusImages')
directorio_coordenadas = os.path.join(directorio_base, 'Contours')
directorio_salida_exp1 = os.path.join(directorio_base, 'ProcessedImages_Disc_Exp1')
directorio_salida_exp2 = os.path.join(directorio_base, 'ProcessedImages_Disc_Exp2')

directorios_salida = {
    'exp1': directorio_salida_exp1,
    'exp2': directorio_salida_exp2
}

# Llamada a la función principal para procesar las imágenes
procesar_imagenes(directorio_imagenes, directorio_coordenadas, directorios_salida)

# Función para verificar imágenes faltantes
def verificar_pacientes_faltantes(directorios_salida, pacientes_ids, ojos=['OD', 'OS']):
    faltantes = {'exp1': [], 'exp2': []}
    for experto, directorio_salida in directorios_salida.items():
        for paciente_id in pacientes_ids:
            for ojo in ojos:
                nombre_archivo = f"RET{paciente_id}{ojo}_reg_disc_{experto}.jpg"
                if not os.path.exists(os.path.join(directorio_salida, nombre_archivo)):
                    faltantes[experto].append(nombre_archivo)
    return faltantes

# Verificar y reportar si faltan imágenes procesadas
faltantes = verificar_pacientes_faltantes(directorios_salida, pacientes_ids)
print("Faltantes para Experto 1:", faltantes['exp1'])
print("Faltantes para Experto 2:", faltantes['exp2'])


Procesando: /content/drive/My Drive/Proyecto de Grado G/FundusImages/RET002OD.jpg con coordenadas de: /content/drive/My Drive/Proyecto de Grado G/Contours/RET002OD_disc_exp1.txt
Imagen guardada en /content/drive/My Drive/Proyecto de Grado G/ProcessedImages_Disc_Exp1: RET002OD_reg_disc_exp1.jpg
Procesando: /content/drive/My Drive/Proyecto de Grado G/FundusImages/RET002OD.jpg con coordenadas de: /content/drive/My Drive/Proyecto de Grado G/Contours/RET002OD_disc_exp2.txt
Imagen guardada en /content/drive/My Drive/Proyecto de Grado G/ProcessedImages_Disc_Exp2: RET002OD_reg_disc_exp2.jpg
Procesando: /content/drive/My Drive/Proyecto de Grado G/FundusImages/RET002OS.jpg con coordenadas de: /content/drive/My Drive/Proyecto de Grado G/Contours/RET002OS_disc_exp1.txt
Imagen guardada en /content/drive/My Drive/Proyecto de Grado G/ProcessedImages_Disc_Exp1: RET002OS_reg_disc_exp1.jpg
Procesando: /content/drive/My Drive/Proyecto de Grado G/FundusImages/RET002OS.jpg con coordenadas de: /content/driv